In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path_individual_provider = './data/Individual_Provider_Network_Data__2021_Quarter_1.csv'
df_individual_provider = pd.read_csv(file_path_individual_provider)

C:\Users\kl3539\AppData\Local\Temp\ipykernel_40224\1027244030.py:2: DtypeWarning: Columns (3,58,60,61,62,63,64,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_individual_provider = pd.read_csv(file_path_individual_provider)


In [4]:
print(df_individual_provider.columns)

Index(['Last Name', 'First Name', 'National Provider Identifier',
       'Medicaid Provider Identification Number', 'Site Name', 'Room or Suite',
       'Street Address', 'Town/City', 'County', 'Borough/County Code',
       ...
       'NYSoH Medical Network  8 Panel Size',
       'NYSoH Medical Network  9 Panel Size', 'SIP-PL Panel Size',
       'FIDA IDD Panel Size', 'HARP Panel Size', 'HIV SNP Panel Size',
       'Site NPI', 'Plan Name', 'PNDS ID', 'Quarter/Year'],
      dtype='object', length=133)


In [6]:
# filter primary care providers (primary designation code 1 or 3)
pcp_filter = (df_individual_provider["Primary designation"] == 1) | (df_individual_provider["Primary designation"] == 3)
df_pcp = df_individual_provider.loc[pcp_filter]

# filter primary care providers who serve Medicaid Enrollees
pcp_medicaid_filter = df_pcp["Medicaid Provider Indicator"] == 1
df_pcp_medicaid = df_pcp.loc[pcp_medicaid_filter]

# filter primary care providers who serve Commercial Non-MCO Medical plan Enrollees
indicator_columns = [
    "Commercial Provider Indicator",
    "Commercial Non-MCO Medical Indicator 1",
    "Commercial Non-MCO Medical Indicator 2",
    "Commercial Non-MCO Medical Indicator 3",
    "Commercial Non-MCO Medical Indicator 4",
    "Commercial Non-MCO Medical Indicator 5",
    "Commercial Non-MCO Medical Indicator 6",
    "Commercial Non-MCO Medical Indicator 7",
    "Commercial Non-MCO Medical Indicator 8"
]
pcp_commercial_filter = pd.Series([False] * len(df_pcp), index=df_pcp.index)
for column in indicator_columns:
    pcp_commercial_filter |= (df_pcp[column] == 1)
df_pcp_commercial = df_pcp.loc[pcp_commercial_filter]

# Get the columns to keep 
columns_to_keep = ['National Provider Identifier', 'Last Name', 'First Name', 'Site Name', 'Street Address', 'Town/City', 'County', 'Borough/County Code', 'Latitude', 'Longitude', 'Primary designation', 'Provider Type', 'Primary Specialty', 'Secondary Specialty']
df_pcp = df_pcp[columns_to_keep]
df_pcp_medicaid = df_pcp_medicaid[columns_to_keep]
df_pcp_commercial = df_pcp_commercial[columns_to_keep]

# print shapes of pcp by serving insurance types
print(f"pcp:{df_pcp.shape}")
print(f"pcp_medicaid: {df_pcp_medicaid.shape}")
print(f"pcp_commercial: {df_pcp_commercial.shape}")

pcp:(1561337, 14)
pcp_medicaid: (130672, 14)
pcp_commercial: (1353388, 14)


In [7]:
# export to csv files
df_pcp.to_csv('pcp_2021.csv', index=False)
df_pcp_medicaid.to_csv('pcp_medicaid_2021.csv', index=False)
df_pcp_commercial.to_csv('pcp_commercial_2021.csv', index=False)